# Getting Started with xRHEED

This notebook shows how to use xRHEED to load, inspect, and plot RHEED data.

In [ ]:
import xrheed
import matplotlib.pyplot as plt

## Load a RHEED data file

Replace `'your_data_file.dat'` with your file path.

In [ ]:
data = xrheed.load_rheed('your_data_file.dat')
data

## Plot the data

In [ ]:
data.plot()
plt.show()